## Exploring the Ethos dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = ""

#### Download the dataset

In [2]:
from autolabel import get_data

get_data('figure_extraction')

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [3]:
import json

from autolabel import LabelingAgent

In [4]:
# load the config
with open('config_figure_extraction.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `attribute_extraction` (since it's an attribute extraction task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at classifying hate speech and identifying...` (how we describe the task to the LLM)
* `prompt.attributes`: `[
            {
                "name": "violence",
                ...
            },
            {
                "name": "directed_vs_generalized",
                ...
            },
            {
                "name": "gender",
                ...
            }
        ]` (the full list of labels to choose from)
* `prompt.few_shot_num`: 5 (how many labeled examples to provide to the LLM)

In [5]:
config

{'task_name': 'EthosAttributeExtraction',
 'task_type': 'attribute_extraction',
 'dataset': {'text_column': 'text',
  'delimiter': ',',
  'image_url_column': 'image_url'},
 'model': {'provider': 'openai_vision', 'name': 'gpt-4-vision-preview'},
 'prompt': {'task_guidelines': 'You are an expert at understanding plots and charts. Your job is to extract the following attributes from the provided image.',
  'attributes': [{'name': 'caption',
    'description': 'The caption associated with the provided plot, usually at the bottom of the image.'},
   {'name': 'x_axis_name',
    'description': 'The label given to the x axis of the plot'},
   {'name': 'y_axis_name',
    'description': 'The label given to the y axis of the plot'},
   {'name': 'title', 'description': 'The title of the provided plot'}],
  'example_template': 'Text: {text}\nOutput: {output_dict}'}}

In [6]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [7]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset
ds = AutolabelDataset("data/figure_extraction/test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.0184 │
│ Number of Examples       │ 1       │
│ Average cost per example │ $0.0184 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

{"text": "You are an expert at understanding plots and charts. Your job is to extract the following attributes from
the provided image.\n\nYou will return the extracted attributes as a json with the following keys:\n{\n    
\"caption\": \"The caption associated with the provided plot, usually at the bottom of the image.\",\n    
\"x_axis_name\": \"The label given to the x axis of the plot\",\n    \"y_axis_name\": \"The label given to the y 
axis of the plot\",\n    \"title\": \"The title of the provided plot\"\n}\n\nNow I want you to label the following 
example:\nText:  and title from the plot\nOutput: ", "image_url": 
"https://autolabel-benchmarking.s3.us-west-2.amazonaws.com/figure_extraction/image0.png"}

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

Actual Cost: 0.0

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ captio… ┃ capti… ┃ captio… ┃ x_axi… ┃ x_axis… ┃ x_axi… ┃ y_axis… ┃ y_axi… ┃ y_axis… ┃ title… ┃ title:… ┃ title… ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 1       │ 1.0    │ 1.0     │ 1      │ 1.0     │ 1.0    │ 1       │ 1.0    │ 1.0     │ 1      │ 1.0     │ 1.0    │
└─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┘

In [9]:
for col in ds.df.columns:
    print(col, ds.df.iloc[0][col])

text  and title from the plot
caption Figure 5: Comparisons between our multifidelity learning paradigm and single low-fidelity (all GPT-3.5) annotation on four domain-specific tasks given the same total 1000 annotation budget. Note that the samples for all GPT-3.5 are drawn based on the uncertainty score.
x_axis_name Annotation size
y_axis_name F1-score
title FPB (Financial)
image_url https://autolabel-benchmarking.s3.us-west-2.amazonaws.com/figure_extraction/image0.png
EthosAttributeExtraction_task_label {'caption': 'Figure 5: Comparisons between our multifidelity learning paradigm and single low-fidelity (all GPT-3.5) annotation on four domain-specific tasks given the same total 1000 annotation budget. Note that the samples for all GPT-3.5 are drawn based on the uncertainty score.', 'x_axis_name': 'Annotation size', 'y_axis_name': 'F1-score', 'title': 'FPB (Financial)'}
caption_label Figure 5: Comparisons between our multifidelity learning paradigm and single low-fidelity (all GPT-3